In [ ]:
import numpy as np 
import pandas as pd
from pandas_datareader import data as pdr 
import pandas_datareader as web
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import mplfinance as mpf
# Market Data 
import yfinance as yf
from datetime import date
from datetime import datetime as dt
from datetime import timedelta
from datetime import datetime
import datetime
import pandas_ta as ta
# Graphing / Visualization 
import plotly.graph_objs as go 

In [ ]:
yf.pdr_override()

In [ ]:
#MACRO DATE FOR PREVIOUS DAY REPORT

for di in range(3,6+1):
    headcheck = 0
    tg_day=di
    tg_month=10
    tg_year=2022
    stock_date = datetime.datetime(tg_year, tg_month, tg_day)
    today=0
    print(stock_date)
    if(today==0):
        if(int(stock_date.strftime("%w"))==1):
            delta_day=3
        else:
            delta_day=1
    else:
        delta_day = 0
    d = stock_date - timedelta(days=delta_day)
    
    stock_analysis_path="Stock Analysis Result/add- StochRSI, "+d.strftime("%Y-%m-%d")+ " Analysis.csv"
    
    #Importing Listed Companies data
    #C:\Users\Abhishek Rao\ML Project\Beta-ML-Project-master\Beta-ML-Project-master\ALL NSE STOCKS
    lcd = pd.read_csv('file:///C:/Users/Abhishek%20Rao/ML%20Project/Beta-ML-Project-master/Beta-ML-Project-master/ALL%20NSE%20STOCKS/LC DATA.csv')
    
    lcd
    
    #Taking the Symbols from the dataframe to list
    company_list=[]
    company_list = list(lcd['Symbol'])
    #print(company_list)
    
    #MACRO
    START_SL = 0
    END_SL = 1890
    SOUND = 0
    
    #THE GRAND LOOP 
    header = pd.DataFrame(columns=['Date','Stock','Combined Factor'])
    for si in range(START_SL,END_SL):
        stock_symbol=company_list[si]
        stock_symbol+='.NS'
        print(stock_symbol)
        STOCK=stock_symbol
        PERIOD='60d'
        INTERVAL='15m'

        df = yf.download(tickers=STOCK, start_date=d ,period=PERIOD, interval=INTERVAL)

        df.index
    
        if(len(df)==0):
            print("Stock "+STOCK+" Skipped due to No Data Found")
            continue
        



        # Get the 200 EMA of the closing price
        ema200 = df['Close'].ewm(span=200, adjust=False, min_periods=200).mean()

        # Get the 26-day EMA of the closing price
        k = df['Close'].ewm(span=12, adjust=False, min_periods=12).mean()
        # Get the 12-day EMA of the closing price
        d = df['Close'].ewm(span=26, adjust=False, min_periods=26).mean()
        # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
        macd = k - d
        # Get the 9-Day EMA of the MACD for the Trigger line
        macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
        # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
        macd_h = macd - macd_s
        # Add all of our new values for the MACD to the dataframe
        df['Macd'] = df.index.map(macd)
        df['Macd histogram'] = df.index.map(macd_h)
        df['Macd Signal'] = df.index.map(macd_s)
        df['200 EMA'] = df.index.map(ema200)
    

        df
    
        #Get Supertrend Data
        #print(ta.supertrend(high=df['High'],low=df['Low'],close=df['Close'],period=12,multiplier=3))
        try:
            df['Supertrend'] = ta.supertrend(high=df['High'],low=df['Low'],close=df['Close'],period=12,multiplier=3)['SUPERT_7_3.0']
        except:
            print("Stock "+STOCK+" skipped Due to Insufficient Data")
            continue
        #print(df)
    
        #Get Stochastic RSI Data
        #print(ta.stochrsi(df['Close'],k=3,d=3, length=14,rsi_length=14))
        try:
            df['Stochastic RSI']=ta.stochrsi(df['Close'],k=3,d=3, length=14,rsi_length=14)['STOCHRSId_14_14_3_3']
        except:
            print("Stock "+STOCK+" skipped Due to Insufficient Data")
            continue
        
        #print(df)
    
        #mpf.plot(df[-100:], figratio=(50,25), type='candle', title=STOCK+' Price', volume=True,tight_layout=True,style='yahoo')
        #Get 200 DEMA
    
        dema200 = ema200*2 - ema200.ewm(span=200, adjust=False, min_periods=200).mean()
        df['200 DEMA'] = df.index.map(dema200)
    

        d = stock_date - timedelta(days=delta_day)
        print(d)
        start_date = datetime.datetime(int(d.strftime("%Y")), int(d.strftime("%m")), int(d.strftime("%d")), 14, 45,0)
        #print(start_date)
        end_date = datetime.datetime(int(d.strftime("%Y")), int(d.strftime("%m")), int(d.strftime("%d")), 15, 15,0)
        #print(end_date)

        #Characteristics analysis
        val_data = df.loc[start_date:end_date]

        val_data

        #GREEN OR RED

        result=[]


        negative_count=0
        positive_count=0
        for i in range(len(val_data)):
            if(val_data['Macd histogram'][i]<0):
                negative_count+=1
            elif(val_data['Macd histogram'][i]>0):
                positive_count+=1

        if(positive_count>negative_count):
            result.append("Green Histogram")
        else:
            result.append("Red Histogram")
    


        #Above or Below 200EMA
        cc=0
        for i in range(len(val_data)):
            if(val_data['Close'][i]>val_data['200 EMA'][i]):
                cc=1
                break
        if(cc==1):
            result.append("Price Above 200 EMA")
        else:
            result.append("Price Below 200 EMA")
    
    

        #Macd Intersecting Above or Intersecting Below or Expanding Above or Expanding Below
        def is_df_ascend_sorted(arr):
            c=0
            for i in range(len(arr)-1):
                if(abs(arr[i])>abs(arr[i+1])):
                    c=1
            if(c==1):
                return False
            else:
                return True

        def is_df_descend_sorted(arr):
            c=0
            for i in range(len(arr)-1):
                if(abs(arr[i])<abs(arr[i+1])):
                    c=1
            if(c==1):
                return False
            else:
                return True
            
            
    
        try:
            if(is_df_ascend_sorted(val_data['Macd histogram'])):
                if(val_data['Macd'][0]<0 or val_data['Macd Signal'][0]<0):
                    result.append("MACD Expanding Below")
                else:
                    result.append("MACD Expanding Above")
            elif(is_df_descend_sorted(val_data['Macd histogram'])):
                if(val_data['Macd'][0]<0 or val_data['Macd Signal'][0]<0):
                    result.append("MACD Intersecting Below")
                else:
                    result.append("MACD Intersecting Above")
            
            else:
                if(val_data['Macd'][0]<0 or val_data['Macd Signal'][0]<0):
                    result.append("MACD Parallel Below")
                else:
                    result.append("MACD Parallel Above")
        except:
            print("Stock "+STOCK+" skipped Due to Insufficient Data")
            continue
        
        
        
        

        #EMA Decreasing or EMA Increasing
        if(is_df_ascend_sorted(val_data['200 EMA'])):
            result.append("EMA Increasing")
        else:
            result.append("EMA Decreasing")

        #Red Supertrend or Green Supertrend
    
        if(int(val_data['Supertrend'][-1:])<int(val_data['Close'][-1:])):
            result.append('Green Supertrend')
        else:
            result.append('Red Supertrend')
    
        #Price Above 200 DEMA or Price Below 200 DEMA
        try:
            if(int(val_data['200 DEMA'][-1:])>int(val_data['Close'][-1:])):
                result.append("Price Below 200 DEMA")
            else:
                result.append("Price Above 200 DEMA")
        except:
            print("Stock "+STOCK+" skipped Due to Insufficient Data")
            continue
    
        #200 EMA Above 200 DEMA Expanding or 200 EMA Above 200 DEMA Intersecting or 200 EMA Below 200 DEMA Expanding or 200 EMA Below 200 DEMA Intersecting
            
        if(int(val_data['200 EMA'][-1:])>int(val_data['200 DEMA'][-1:])):
            if(is_df_ascend_sorted(val_data['200 EMA']-val_data['200 DEMA'])):
                result.append("200 EMA Above 200 DEMA Expanding")
            else:
                result.append("200 EMA Above 200 DEMA Intersecting")
        else:
            if(is_df_ascend_sorted(val_data['200 EMA']-val_data['200 DEMA'])):
                result.append("200 EMA Below 200 DEMA Expanding")
            else:
                result.append("200 EMA Below 200 DEMA Intersecting")

        #O2C Profit or O2C Loss
        try:
            if(float(df['Close'][-1:])>float(df['Open'][-18:-17])):
                result.append("O2C Profit")
            else:
                result.append("O2C Loss")
        except:
            print("Stock "+STOCK+" skipped Due to Insufficient Data")
            continue

        #Stochastic RSI Increasing or Stochastic RSI Decreasing

        try:
            if(is_df_ascend_sorted(val_data['Stochastic RSI'])):
                result.append("Stochastic RSI Increasing")
            else:
                result.append("Stochastic RSI Decreasing")
        except:
            print("Stock "+STOCK+" skipped Due to Insufficient Data")
            continue




        print(si, end=". ")
        print(STOCK,end=": ")
        print(result,end="\n\n")




        combined_factor = ', '.join(map(str, result))
        header = header.append({'Date' : d.strftime("%Y-%m-%d"), 'Stock' :STOCK, 'Combined Factor' : combined_factor},ignore_index=True)
   

    target_filename = stock_analysis_path
    header.to_csv(target_filename)

    pd.read_csv(target_filename)